In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=30

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "datasets2",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2349 files belonging to 2 classes.


In [4]:
class_names = dataset.class_names
class_names

['non-plant', 'plant']

In [5]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1]


In [6]:
len(dataset)

74

In [7]:
train_size = 0.7
len(dataset)*train_size

51.8

In [8]:
train_ds = dataset.take(51)
len(train_ds)

51

In [9]:
test_ds = dataset.skip(51)
len(test_ds)

23

In [10]:
val_size=0.15
len(dataset)*val_size

11.1

In [11]:
val_ds = test_ds.take(11)
len(val_ds)

11

In [12]:
test_ds = test_ds.skip(11)
len(test_ds)

12

In [13]:
def get_dataset_partitions_tf(ds, train_split=0.7, val_split=0.15, test_split=0.15, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [15]:
len(train_ds)

51

In [16]:
len(val_ds)

11

In [17]:
len(test_ds)

12

In [18]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [19]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [20]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 2

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 256, 256, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (32, 254, 254, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 127, 127, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 125, 125, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 62, 62, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 60, 60, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (32, 30, 30, 64)         

In [22]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [23]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=30,
)

Epoch 1/30
51/51 [==============================] - 184s 3s/step - loss: 0.3245 - accuracy: 0.8171 - val_loss: 0.0297 - val_accuracy: 0.9915
Epoch 2/30
51/51 [==============================] - 96s 2s/step - loss: 0.0480 - accuracy: 0.9857 - val_loss: 0.0178 - val_accuracy: 0.9886
Epoch 3/30
51/51 [==============================] - 96s 2s/step - loss: 0.0210 - accuracy: 0.9938 - val_loss: 0.0062 - val_accuracy: 0.9972
Epoch 4/30
51/51 [==============================] - 104s 2s/step - loss: 0.0128 - accuracy: 0.9969 - val_loss: 0.0473 - val_accuracy: 0.9886
Epoch 5/30
51/51 [==============================] - 102s 2s/step - loss: 0.0291 - accuracy: 0.9901 - val_loss: 0.0095 - val_accuracy: 0.9943
Epoch 6/30
51/51 [==============================] - 100s 2s/step - loss: 0.0214 - accuracy: 0.9926 - val_loss: 0.0521 - val_accuracy: 0.9744
Epoch 7/30
51/51 [==============================] - 102s 2s/step - loss: 0.0133 - accuracy: 0.9975 - val_loss: 0.0064 - val_accuracy: 0.9972
Epoch 8/30
51/5

In [24]:
scores = model.evaluate(test_ds)

12/12 [==============================] - 52s 616ms/step - loss: 4.3846e-06 - accuracy: 1.0000


In [25]:
import os
model.save(f"C:/Users/ASUS/Documents/models/modelPreProcess")

INFO:tensorflow:Assets written to: C:/Users/ASUS/Documents/models/modelPreProcess\assets
